# Pytorch_9_Proste sieci neuronowe_pytorch-custom-nn-modules
https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules


In [1]:
import torch

Odpalam karte graficzną GPU

In [2]:
device = torch.device('cpu') # obliczenia robie na CPU
#device = torch.device('cuda') # obliczenia robie na GPU

### Dane
- N to wielkość partii; 
- D_in jest wymiarem wejściowym; 
- H jest ukrytym wymiarem; 
- D_out jest wymiarem wyjściowym. 


## <span style="color:red">Przyjmuje 10 zmiennych wynikowych! Coś nowego!</span>

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

Powyżej zostały określone parametry aby tensor zmiennych niezależnych i tensor wynikowy były odpowiednie 

x: 64 obserwacji i 1000 zmiennych

y: 64 obserwacji i 10 zmiennych

### Tworzy losowe dane wejściowe i wyjściowe 

In [4]:
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [5]:
x.shape

torch.Size([64, 1000])

In [6]:
print('x ',x.size())
print('y ',y.size())

x  torch.Size([64, 1000])
y  torch.Size([64, 10])


### Niestandardowe funkcje autograd

In [7]:
class MyReLU(torch.autograd.Function):
  """
   Możemy zaimplementować własne niestandardowe funkcje autograd poprzez podklasowanie
   torch.autograd. Funkcja i realizacja przejść do przodu i wstecz
   które działają na Tensorach.
  """
  
  def forward(ctx, x):
    """
     W przekazie do przodu otrzymujemy obiekt kontekstowy i Tensor zawierający
     Wejście; musimy zwrócić Tensor zawierający dane wyjściowe i możemy użyć
     obiekt kontekstowy do buforowania obiektów do użycia w przebiegu wstecznym.
    """
    ctx.save_for_backward(x)
    return x.clamp(min=0)

  
  def backward(ctx, grad_output):
    """
     W wstecznym przebiegu otrzymujemy obiekt kontekstu i Tensor zawierający
     gradient straty w stosunku do produkcji wytworzonej podczas
     podanie do przodu. Możemy pobrać buforowane dane z obiektu kontekstu i musimy
     obliczyć i zwrócić gradient straty w odniesieniu do danych wejściowych do
     funkcja przewijania do przodu.
    """
    x, = ctx.saved_tensors
    grad_x = grad_output.clone()
    grad_x[x < 0] = 0
    return grad_x

### Tworzy wagi losowe dla x i y <span style="color:red">(requires_grad=True)</span>

Utwórzenie losowych tensorów dla wag; ustawienie 'requires_grad=True' oznacza, że my
chce obliczyć gradienty dla tych Tensorów podczas przebiegu wstecznego.

In [8]:
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

In [9]:
w1

tensor([[-0.2364, -1.1935, -0.2067,  ..., -1.5519, -2.0828, -0.4266],
        [-0.8470, -0.4808, -1.9644,  ..., -1.2827,  1.2122, -0.2980],
        [-0.3290,  0.7171, -0.5915,  ..., -1.1769,  0.2304,  1.9914],
        ...,
        [-0.2232,  0.3495, -0.3527,  ..., -2.0763, -0.0472, -1.3370],
        [-1.0502, -0.5856, -0.5225,  ..., -0.9558, -0.2693, -1.6711],
        [ 0.6511, -0.1234,  1.0482,  ..., -0.5457, -0.9574, -1.2464]],
       requires_grad=True)

In [10]:
print('w1 dla x: ', w1.shape)
print('w2 dla y: ', w2.shape)

w1 dla x:  torch.Size([1000, 100])
w2 dla y:  torch.Size([100, 10])


### Definiowanie nauki
     Przekaz do przodu: oblicz przewidywane y przy użyciu operacji na Tensorach; nazywamy nasz
     niestandardowa implementacja ReLU za pomocą funkcji MyReLU.apply


In [11]:
learning_rate = 1e-6
for t in range(500):
 

    y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
      # Funkcja błędu r2 dla drukowana dla każdej epoki
    loss = (y_pred - y).pow(2).sum()
    print('Epoka:',t, loss.item())
    
       #KROK 1
       # Użyj programu autograd, aby obliczyć przebieg wsteczny. 
    loss.backward()

       #KROK 2
       # Zaktualizuj wagi przy użyciu spadku gradientu.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

       #KROK 3
       # Ręcznie zeruj gradienty po przejściu do tyłu
        w1.grad.zero_()
        w2.grad.zero_()
    
    

Epoka: 0 32798832.0
Epoka: 1 30828960.0
Epoka: 2 30828920.0
Epoka: 3 28419262.0
Epoka: 4 22279720.0
Epoka: 5 14654725.0
Epoka: 6 8478973.0
Epoka: 7 4705764.5
Epoka: 8 2732645.0
Epoka: 9 1745335.75
Epoka: 10 1234010.125
Epoka: 11 944412.875
Epoka: 12 761477.3125
Epoka: 13 633695.875
Epoka: 14 537585.375
Epoka: 15 461874.40625
Epoka: 16 400091.6875
Epoka: 17 348715.5
Epoka: 18 305467.09375
Epoka: 19 268808.6875
Epoka: 20 237432.796875
Epoka: 21 210449.59375
Epoka: 22 187127.296875
Epoka: 23 166895.9375
Epoka: 24 149251.40625
Epoka: 25 133812.3125
Epoka: 26 120255.71875
Epoka: 27 108307.5859375
Epoka: 28 97752.953125
Epoka: 29 88401.265625
Epoka: 30 80099.6015625
Epoka: 31 72706.453125
Epoka: 32 66112.125
Epoka: 33 60210.703125
Epoka: 34 54921.2265625
Epoka: 35 50166.9609375
Epoka: 36 45885.328125
Epoka: 37 42023.1875
Epoka: 38 38531.98828125
Epoka: 39 35370.53515625
Epoka: 40 32503.40234375
Epoka: 41 29899.84375
Epoka: 42 27534.64453125
Epoka: 43 25381.53515625
Epoka: 44 23418.87109375
E

Epoka: 292 0.022348644211888313
Epoka: 293 0.021259430795907974
Epoka: 294 0.02023790031671524
Epoka: 295 0.01925421506166458
Epoka: 296 0.01832660287618637
Epoka: 297 0.01744696870446205
Epoka: 298 0.01659097522497177
Epoka: 299 0.015796082094311714
Epoka: 300 0.015035102143883705
Epoka: 301 0.014302595518529415
Epoka: 302 0.01362520083785057
Epoka: 303 0.012964709661900997
Epoka: 304 0.012351114302873611
Epoka: 305 0.011754434555768967
Epoka: 306 0.011190414428710938
Epoka: 307 0.01065540499985218
Epoka: 308 0.01015055924654007
Epoka: 309 0.009673358872532845
Epoka: 310 0.00921807624399662
Epoka: 311 0.008782424032688141
Epoka: 312 0.008363638073205948
Epoka: 313 0.007968978025019169
Epoka: 314 0.007595540024340153
Epoka: 315 0.007235419005155563
Epoka: 316 0.0069013661704957485
Epoka: 317 0.006585077382624149
Epoka: 318 0.006272567436099052
Epoka: 319 0.00598646467551589
Epoka: 320 0.005714692175388336
Epoka: 321 0.005444324109703302
Epoka: 322 0.005194548517465591
Epoka: 323 0.0049